In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import scipy.stats as st
import covid # Custom module that provides monthly the number of COVID19 confirmed cases
import ecom  # Custom module that provides Canada and South Korea's sales data - monthly, annual

In [2]:
# Read the US Data
us_merged_df = pd.read_csv("data/dtype_merge.csv")

In [3]:
us_merged_df.head()

,per_idx,cat_idx,dt_idx,et_idx,geo_idx,is_adj,val,cat_code,cat_desc,cat_indent,per_name,dt_code,dt_desc,dt_unit,month,year,mon/yy
0,1,5,1,0,1,0,130683.0,44000,44000: Retail Trade,0,Jan-92,SM,Sales - Monthly,MLN$,Jan,92,Jan/92
1,1,5,1,0,1,1,146925.0,44000,44000: Retail Trade,0,Jan-92,SM,Sales - Monthly,MLN$,Jan,92,Jan/92
2,1,5,2,0,1,0,235509.0,44000,44000: Retail Trade,0,Jan-92,IM,Inventories - Monthly,MLN$,Jan,92,Jan/92
3,1,5,2,0,1,1,242006.0,44000,44000: Retail Trade,0,Jan-92,IM,Inventories - Monthly,MLN$,Jan,92,Jan/92
4,1,5,3,0,1,0,1.8,44000,44000: Retail Trade,0,Jan-92,IR,Inventories/Sales Ratio,RATIO,Jan,92,Jan/92


In [4]:
# Get non-adjusted monthly overall retail sales data from the merged data
us_retail_df = us_merged_df.loc[(us_merged_df.loc[:, "is_adj"] == 0) &
                                (us_merged_df.loc[:, "cat_code"] == "44W72") &
                                (us_merged_df.loc[:, "dt_code"] == "SM")]

us_retail_df = us_retail_df.drop(columns = ["per_idx", "cat_idx", "dt_idx", "et_idx", "geo_idx", "is_adj", "cat_code",
                                            "cat_desc", "cat_indent", "per_name", "dt_code", "dt_desc"])

us_retail_df.head()

NameError: name 'us_sales_df' is not defined

In [ ]:
# Get non-adjusted monthly e-commerce sales data from the merged data
us_ecommerce_df = us_merged_df.loc[(us_merged_df.loc[:, "is_adj"] == 0) &
                                   (us_merged_df.loc[:, "cat_code"] == "4541") &
                                   (us_merged_df.loc[:, "dt_code"] == "SM")]

us_ecommerce_df = us_ecommerce_df.drop(columns = ["per_idx", "cat_idx", "dt_idx", "et_idx", "geo_idx", "is_adj", "cat_code", "cat_desc",
                                                  "cat_indent", "per_name", "dt_code", "dt_desc", "dt_unit", "month", "year"])

us_ecommerce_df.head()

In [ ]:
# Merge overall retail and e-commerce data into a dataframe
us_sales_df = pd.merge(us_retail_df, us_ecommerce_df, on = "mon/yy")
us_sales_df = us_sales_df.rename(columns = {"val_x":"Retail Amount", "val_y":"E-Commerce Amount"})
us_sales_df.head()

In [ ]:
# Calculate E-Commerce Share(%)
us_sales_df["E-Commerce Share(%)"] = us_sales_df.iloc[:, 5] / us_sales_df.iloc[:, 0] * 100

In [ ]:
# Calculate overall retail and E-Commerce MoM growth(%), Convert year type %y to %Y
for idx in us_sales_df.index:
    us_sales_df.loc[idx, "year"] = us_sales_df.loc[idx, "year"] + 1900 if us_sales_df.loc[idx, "year"] > 90 else us_sales_df.loc[idx, "year"] + 2000
    
    try:
        us_sales_df.loc[idx, "Retail Growth(%)"] = (us_sales_df.loc[idx, "Retail Amount"] - us_sales_df.loc[idx-1, "Retail Amount"]) / us_sales_df.loc[idx-1, "Retail Amount"] * 100
        us_sales_df.loc[idx, "E-Commerce Growth(%)"] = (us_sales_df.loc[idx, "E-Commerce Amount"] - us_sales_df.loc[idx-1, "E-Commerce Amount"]) / us_sales_df.loc[idx-1, "E-Commerce Amount"] * 100
    except KeyError:
        pass

us_sales_df.head()

In [ ]:
# Annual retail and e-commerce sales data
us_sales_a_df = us_sales_df.iloc[:, [0, 3, 5]].groupby("year").sum()
us_sales_a_df["E-Commerce Share(%)"] = us_sales_a_df.iloc[:, 1] / us_sales_a_df.iloc[:, 0] * 100

for idx in us_sales_a_df.index[1:]:
    us_sales_a_df.loc[idx, "Retail Growth(%)"] = (us_sales_a_df.loc[idx, "Retail Amount"] - us_sales_a_df.loc[idx-1, "Retail Amount"]) / us_sales_a_df.loc[idx-1, "Retail Amount"] * 100
    us_sales_a_df.loc[idx, "E-Commerce Growth(%)"] = (us_sales_a_df.loc[idx, "E-Commerce Amount"] - us_sales_a_df.loc[idx-1, "E-Commerce Amount"]) / us_sales_a_df.loc[idx-1, "E-Commerce Amount"] * 100

this_year, last_year = (us_sales_a_df.tail(2).index[1], us_sales_a_df.tail(2).index[0])
num_this_year = len(us_sales_df.loc[us_sales_df.loc[:, "year"] == this_year].index)

last_retail = us_sales_df.loc[us_sales_df.iloc[:, 3] == last_year].iloc[:num_this_year, 0].sum()
last_online = us_sales_df.loc[us_sales_df.iloc[:, 3] == last_year].iloc[:num_this_year, 5].sum()

us_sales_a_df.loc[this_year, "Retail Growth(%)"] = (us_sales_a_df.tail(1).iloc[0, 0] - last_retail) / last_retail * 100
us_sales_a_df.loc[this_year, "E-Commerce Growth(%)"] = (us_sales_a_df.tail(1).iloc[0, 1] - last_online) / last_online * 100

us_sales_a_df.tail()

In [ ]:
# Remove data in 1992 - 2009
us_sales_a_df = us_sales_a_df.reset_index()
us_sales_df = us_sales_df.loc[us_sales_df.loc[:, "year"] >= 2010]
us_sales_a_df = us_sales_a_df.loc[us_sales_a_df.loc[:, "year"] >= 2010]
us_sales_a_df.head()

In [ ]:
# Plot retail and e-commerce sales line charts for 10 years
x_axis = us_sales_df.loc[:, "mon/yy"]
x_label = list(us_sales_df.loc[:, "year"].unique())
x_label.insert(0, 2010)

y_axis_r = us_sales_df.loc[:, "Retail Amount"]
y_axis_e = us_sales_df.loc[:, "E-Commerce Amount"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis, y_axis_r, color = "r")
ax1.set_title("Overall Retail Sales Trend in the US (Monthly)")
ax1.set_ylabel("Amount (mil$)")
ax1.set_xlim(x_axis.head(1), x_axis.tail(1))
ax1.set_ylim(0, y_axis_r.max()*1.5)
ax1.grid()
ax1.fill_between(x_axis, y_axis_r, color = "r", alpha = 0.5)
ax1.set_xticks(x_axis)

ax1.xaxis.set_major_locator(MultipleLocator(12))
ax1.set_xticklabels(x_label, minor = False)

ax2.plot(x_axis, y_axis_e, color = "b")
ax2.set_title("E-Commerce Sales Trend in the US (Monthly)")
ax2.set_ylabel("Amount (mil$)")
ax2.set_xlim(x_axis.head(1), x_axis.tail(1))
ax2.set_ylim(0, y_axis_e.max()*1.5)
ax2.grid()
ax2.fill_between(x_axis, y_axis_e, color = "b", alpha = 0.5)
ax2.xaxis.set_major_locator(MultipleLocator(12))
ax2.set_xticklabels(x_label, minor = False)

plt.savefig("charts/us_sales.png", dpi = 200)

plt.show()

In [ ]:
ax = us_sales_a_df.iloc[:, 3].plot(kind = "bar", width = 0.8, figsize=(10,4), color = "r", alpha = 0.75)
x_label = us_sales_a_df.iloc[:, 0]

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(round(height, 2))+"%", (left+width/2, height/2), ha='center', color = "white")

plt.title("Annual E-Commerce Share of Retail Sales")
plt.ylabel("(%)")
ax.set_xticklabels(x_label, rotation = 0)
plt.savefig("charts/e-commerce_share.png", dpi = 200)
plt.show()

In [ ]:
# Plot retail and e-commerce sales growth rate for 10 years
ax = us_sales_a_df.iloc[:, [4, 5]].plot(kind = "bar", width = 0.8, figsize=(18,4), color = ["slateblue", "r"], alpha = 0.75)

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(round(height, 2))+"%", (left+width/2, height/2), ha='center', color = "white")

plt.title("Annual Retail & E-Commerce Sales Growth")
plt.axhline(y=0, color='red', linewidth=0.5)
plt.ylabel("(%)")
ax.set_xticklabels(x_label, rotation = 0)
plt.savefig("charts/sales_growth.png", dpi = 200)
plt.show()

In [ ]:
us_sales_1 = us_sales_df.tail(24)
us_sales_1

In [ ]:
# Plot e-commerce share
x_axis_1 = us_sales_1.head(12).loc[:, "mon/yy"]
x_axis_2 = us_sales_1.tail(12).loc[:, "mon/yy"]

y_axis_1 = us_sales_1.head(12).iloc[:, 6]
y_axis_2 = us_sales_1.tail(12).iloc[:, 6]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis_1, y_axis_1, color = "r")
ax1.set_title("E-Commerce Sales Share(%) 06/2018 - 05/2019")
ax1.set_ylabel("Share (%)")
ax1.set_xlim(x_axis_1.head(1), x_axis_1.tail(1))
ax1.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax1.fill_between(x_axis_1, y_axis_1, color = "r", alpha = 0.5)
ax1.set_xticks(x_axis_1)

ax2.plot(x_axis_2, y_axis_2, color = "b")
ax2.set_title("E-Commerce Sales Share(%) 06/2019 - 05/2020")
ax2.set_ylabel("Share (%)")
ax2.set_xlim(x_axis_2.head(1), x_axis_2.tail(1))
ax2.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax2.fill_between(x_axis_2, y_axis_2, color = "b", alpha = 0.5)
ax2.axvline(x=7.22, color='blue', linewidth=1)
ax2.annotate("China confirmed COVID19 on 01/07 →", (2.2, 22), color = "blue")
ax2.axvline(x=7.6, color='red', linewidth=1)
ax2.annotate("First Case Detected in the US on 01/19 →", (1.9, 20), color = "red")
ax2.axvline(x=9.5, color='green', linewidth=1)
ax2.annotate("Lockdown started in the US on 03/15 →", (4.1, 18), color = "green")

plt.savefig("charts/e-commerce_growth.png", dpi = 200)

plt.show()

In [ ]:
# Plot overall retail amount
y_axis_1 = us_sales_1.head(12).iloc[:, 0]
y_axis_2 = us_sales_1.tail(12).iloc[:, 0]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis_1, y_axis_1, color = "r")
ax1.set_title("Overall Retail Sales 06/2018 - 05/2019")
ax1.set_ylabel("Amount ($ 1mil)")
ax1.set_xlim(x_axis_1.head(1), x_axis_1.tail(1))
ax1.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax1.fill_between(x_axis_1, y_axis_1, color = "r", alpha = 0.5)
ax1.set_xticklabels(x_axis_1, rotation = 30)

ax2.plot(x_axis_2, y_axis_2, color = "b")
ax2.set_title("Overall Retail Sales 06/2019 - 05/2020")
ax2.set_ylabel("Amount ($ 1mil)")
ax2.set_xlim(x_axis_2.head(1), x_axis_2.tail(1))
ax2.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax2.fill_between(x_axis_2, y_axis_2, color = "b", alpha = 0.5)
ax2.set_xticklabels(x_axis_2, rotation = 30)
ax2.axvline(x=7.22, color='blue', linewidth=1)
ax2.annotate("China confirmed COVID19 on 01/07 →", (2.2, 650000), color = "blue")
ax2.axvline(x=7.6, color='red', linewidth=1)
ax2.annotate("First Case Detected in the US on 01/19 →", (1.9, 600000), color = "red")
ax2.axvline(x=9.5, color='green', linewidth=1)
ax2.annotate("Lockdown started in the US on 03/15 →", (4.1, 550000), color = "green")

plt.savefig("charts/e-commerce_growth.png", dpi = 200)

plt.show()

In [ ]:
covid_df = covid.get_num_of_cases(["united-states", "canada", "china", "south-korea"])
covid_df

In [ ]:
canada_df = ecom.get_canada_df_m()
canada_a_df = ecom.get_canada_df_y()
korea_df = ecom.get_korea_df_m()
korea_a_df = ecom.get_korea_df_y()
canada_2 = canada_df.tail(24)
korea_2 = korea_df.tail(24)
canada_df.head()

In [ ]:
korea_df.head()

In [ ]:
# Plot retail and e-commerce sales growth rate for 4 years
ax = canada_a_df.iloc[1:, [4, 5]].plot(kind = "bar", width = 0.8, figsize=(10,4), color = ["slateblue", "r"], alpha = 0.75)
x_label = canada_a_df.iloc[1:, 0]

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(round(height, 2))+"%", (left+width/2, height/2), ha='center', color = "black")

plt.title("Annual Retail & Online Sales Growth in Canada")
plt.axhline(y=0, color='red', linewidth=0.5)
plt.ylabel("(%)")
ax.set_xticklabels(x_label, rotation = 0)
plt.savefig("charts/sales_growth_canada.png", dpi = 200)
plt.show()

In [ ]:
# Plot e-commerce share in Canada
y_axis_1 = canada_2.head(12).iloc[:, 7].astype(float)
y_axis_2 = canada_2.tail(12).iloc[:, 7].astype(float)

x_axis_1 = canada_2.head(12).loc[:, "Date"]
x_axis_2 = canada_2.tail(12).loc[:, "Date"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis_1, y_axis_1, color = "r")
ax1.set_title("E-Commerce Sales Share in Canada, 06/2018 - 05/2019")
ax1.set_ylabel("Share (%)")
ax1.set_xlim(x_axis_1.head(1), x_axis_1.tail(1))
ax1.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax1.fill_between(x_axis_1, y_axis_1, color = "r", alpha = 0.5)
ax1.set_xticklabels(x_axis_1, rotation = 30)

ax2.plot(x_axis_2, y_axis_2, color = "b")
ax2.set_title("E-Commerce Sales Share in Canada, 06/2019 - 05/2020")
ax2.set_ylabel("Share (%)")
ax2.set_xlim(x_axis_2.head(1), x_axis_2.tail(1))
ax2.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax2.fill_between(x_axis_2, y_axis_2, color = "b", alpha = 0.5)
ax2.set_xticklabels(x_axis_2, rotation = 30)
ax2.axvline(x=7.22, color='blue', linewidth=1)
ax2.annotate("China confirmed COVID19 on 01/7 →", (2.2, 10), color = "blue")
ax2.axvline(x=7.8, color='red', linewidth=1)
ax2.annotate("First Case Detected in Canada on 01/25 →", (2, 9), color = "red")
ax2.axvline(x=9.54, color='green', linewidth=1)
ax2.annotate("Lockdown started in Canada on 03/17 →", (4.14, 8), color = "green")

plt.savefig("charts/e-commerce_share_canada.png", dpi = 200)

plt.show()

In [ ]:
# Plot overall retail amount
y_axis_1 = canada_2.head(12).iloc[:, 1].astype(float)
y_axis_2 = canada_2.tail(12).iloc[:, 1].astype(float)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis_1, y_axis_1, color = "r")
ax1.set_title("Overall Retail Sales in Canada, 06/2018 - 05/2019")
ax1.set_ylabel("Amount ($ 1mil)")
ax1.set_xlim(x_axis_1.head(1), x_axis_1.tail(1))
ax1.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax1.fill_between(x_axis_1, y_axis_1, color = "r", alpha = 0.5)
ax1.set_xticklabels(x_axis_1, rotation = 30)

ax2.plot(x_axis_2, y_axis_2, color = "b")
ax2.set_title("Overall Retail Sales in Canada, 06/2019 - 05/2020")
ax2.set_ylabel("Amount ($ 1mil)")
ax2.set_xlim(x_axis_2.head(1), x_axis_2.tail(1))
ax2.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax2.fill_between(x_axis_2, y_axis_2, color = "b", alpha = 0.5)
ax2.set_xticklabels(x_axis_2, rotation = 30)
ax2.axvline(x=7.22, color='blue', linewidth=1)
ax2.annotate("China confirmed COVID19 on 01/7 →", (2.2, 50000), color = "blue")
ax2.axvline(x=7.8, color='red', linewidth=1)
ax2.annotate("First Case Detected in Canada on 01/25 →", (2, 47000), color = "red")
ax2.axvline(x=9.54, color='green', linewidth=1)
ax2.annotate("Lockdown started in Canada on 03/17 →", (4.14, 44000), color = "green")


plt.savefig("charts/retail_sales_canada.png", dpi = 200)

plt.show()

In [ ]:
# Plot retail and e-commerce sales growth rate for 3 years
ax = korea_a_df.iloc[1:, [3, 4]].plot(kind = "bar", width = 0.8, figsize=(8,4), color = ["slateblue", "r"], alpha = 0.75)
x_label = korea_a_df.iloc[1:, 0]

for p in ax.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax.annotate(str(round(height, 2))+"%", (left+width/2, height/2), ha='center', color = "black")

plt.title("Annual Retail & Online Sales Growth in Korea")
plt.axhline(y=0, color='red', linewidth=0.5)
plt.ylabel("(%)")
ax.set_xticklabels(x_label, rotation = 0)
plt.savefig("charts/sales_growth_korea.png", dpi = 200)
plt.show()

In [ ]:
# Plot e-commerce share in Korea
y_axis_1 = korea_2.head(12).iloc[:, 5].astype(float)
y_axis_2 = korea_2.tail(12).iloc[:, 5].astype(float)

x_axis_1 = korea_2.head(12).loc[:, "Date"]
x_axis_2 = korea_2.tail(12).loc[:, "Date"]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis_1, y_axis_1, color = "r")
ax1.set_title("E-Commerce Sales Share in Korea, 06/2018 - 05/2019")
ax1.set_ylabel("Share (%)")
ax1.set_xlim(x_axis_1.head(1), x_axis_1.tail(1))
ax1.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax1.fill_between(x_axis_1, y_axis_1, color = "r", alpha = 0.5)
ax1.set_xticklabels(x_axis_1, rotation = 30)

ax2.plot(x_axis_2, y_axis_2, color = "b")
ax2.set_title("E-Commerce Sales Share in Korea, 06/2019 - 05/2020")
ax2.set_ylabel("Share (%)")
ax2.set_xlim(x_axis_2.head(1), x_axis_2.tail(1))
ax2.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax2.fill_between(x_axis_2, y_axis_2, color = "b", alpha = 0.5)
ax2.set_xticklabels(x_axis_2, rotation = 30)
ax2.axvline(x=7.22, color='blue', linewidth=1)
ax2.annotate("China confirmed COVID19 on 01/7 →", (2.2, 40), color = "blue")
ax2.axvline(x=7.65, color='red', linewidth=1)
ax2.annotate("First Case Detected in Korea on 01/20 →", (2, 35), color = "red")

plt.savefig("charts/e-commerce_share_korea.png", dpi = 200)

plt.show()

In [ ]:
# Plot overall retail amount
y_axis_1 = korea_2.head(12).iloc[:, 1].astype(float)
y_axis_2 = korea_2.tail(12).iloc[:, 1].astype(float)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4))

ax1.plot(x_axis_1, y_axis_1, color = "r")
ax1.set_title("Overall Retail Sales in Korea, 06/2018 - 05/2019")
ax1.set_ylabel("Amount ($ 1mil)")
ax1.set_xlim(x_axis_1.head(1), x_axis_1.tail(1))
ax1.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax1.fill_between(x_axis_1, y_axis_1, color = "r", alpha = 0.5)
ax1.set_xticklabels(x_axis_1, rotation = 30)

ax2.plot(x_axis_2, y_axis_2, color = "b")
ax2.set_title("Overall Retail Sales in Korea, 06/2019 - 05/2020")
ax2.set_ylabel("Amount ($ 1mil)")
ax2.set_xlim(x_axis_2.head(1), x_axis_2.tail(1))
ax2.set_ylim(y_axis_2.min()*0.7, y_axis_2.max()*1.3)
ax2.fill_between(x_axis_2, y_axis_2, color = "b", alpha = 0.5)
ax2.set_xticklabels(x_axis_2, rotation = 30)
ax2.axvline(x=7.22, color='blue', linewidth=1)
ax2.annotate("China confirmed COVID19 on 01/7 →", (2.2, 40000), color = "blue")
ax2.axvline(x=7.65, color='red', linewidth=1)
ax2.annotate("First Case Detected in Korea on 01/20 →", (2, 37000), color = "red")

plt.savefig("charts/retail_sales_korea.png", dpi = 200)

plt.show()

In [ ]:
covid_sales_df = covid_df.loc[(covid_df.loc[:, "Country"] == "United States of America") |
                              (covid_df.loc[:, "Country"] == "Canada") |
                              (covid_df.loc[:, "Country"] == "Korea (South)")]
covid_sales_df

In [ ]:
covid_sales_us = covid_sales_df.iloc[:5, :]
covid_sales_ca = covid_sales_df.iloc[5:10, :]
covid_sales_kr = covid_sales_df.iloc[10:15, :]

covid_merge_df = pd.merge(covid_sales_us, covid_sales_ca, on = "Date")
covid_merge_df = pd.merge(covid_merge_df, covid_sales_kr, on = "Date")
covid_merge_df = covid_merge_df.rename(columns = {"New Cases_x":"US Cases", "New Cases_y":"CA Cases", "New Cases":"KR Cases",
                                                  "Cumulative Cases_x":"US Cumulative Cases", "Cumulative Cases_y":"US Cumulative Cases",
                                                  "Cumulative Cases":"KR Cumulative Cases", "E-Commerce Growth(%)_x":"US E-Commerce Growth(%)",
                                                  "E-Commerce Growth(%)_y":"CA E-Commerce Growth(%)", "E-Commerce Growth(%)":"KR E-Commerce Growth(%)",
                                                  "E-Commerce Share(%)_x":"US E-Commerce Share(%)", "E-Commerce Share(%)_y":"CA E-Commerce Share(%)",
                                                  "E-Commerce Share(%)":"KR E-Commerce Share(%)", "New Cases per 100000_x":"US New Cases per 100k",
                                                  "New Cases per 100000_y":"CA New Cases per 100k", "New Cases per 100000":"KR New Cases per 100k"})
                                        
covid_merge_df

In [ ]:
x_label = list(covid_merge_df.iloc[:, 1])

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 12))
trace1 = ax1.bar(range(len(x_label)), covid_merge_df.iloc[:, 2], width = 0.8, color = "slateblue", alpha = 0.75)
trace2 = ax2.bar(range(len(x_label)), covid_merge_df.iloc[:, 9], width = 0.8, color = "r", alpha = 0.75)
trace3 = ax3.bar(range(len(x_label)), covid_merge_df.iloc[:, 16], width = 0.8, color = "pink", alpha = 0.75)

for p in ax1.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax1.annotate(int(height), (left+width/2, height), ha='center', color = "black")

for p in ax2.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax2.annotate(int(height), (left+width/2, height), ha='center', color = "black")

for p in ax3.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax3.annotate(int(height), (left+width/2, height), ha='center', color = "black")

ax1.set_title("New Confirmed Cases in the US")
ax2.set_title("New Confirmed Cases in Canada")
ax3.set_title("New Confirmed Cases in South Korea")

ax1.set_ylabel("Cases")
ax2.set_ylabel("Cases")
ax3.set_ylabel("Cases")

ax1.set_ylim(0, covid_merge_df.iloc[:, 2].max()*1.3)
ax2.set_ylim(0, covid_merge_df.iloc[:, 9].max()*1.5)
ax3.set_ylim(0, covid_merge_df.iloc[:, 16].max()*1.8)

ax1.set_xticks(range(len(x_label)))
ax2.set_xticks(range(len(x_label)))
ax3.set_xticks(range(len(x_label)))

ax1.set_xticklabels(x_label, rotation = 0)
ax2.set_xticklabels(x_label, rotation = 0)
ax3.set_xticklabels(x_label, rotation = 0)

plt.savefig("charts/confirmed_cases.png", dpi = 200)
plt.show()

In [ ]:
x_label = list(covid_merge_df.iloc[:, 1])

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 12))
trace1 = ax1.bar(range(len(x_label)), covid_merge_df.iloc[:, 6], width = 0.8, color = "slateblue", alpha = 0.75)
trace2 = ax2.bar(range(len(x_label)), covid_merge_df.iloc[:, 13], width = 0.8, color = "r", alpha = 0.75)
trace3 = ax3.bar(range(len(x_label)), covid_merge_df.iloc[:, 20], width = 0.8, color = "pink", alpha = 0.75)

for p in ax1.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax1.annotate(int(height), (left+width/2, height), ha='center', color = "black")

for p in ax2.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax2.annotate(int(height), (left+width/2, height), ha='center', color = "black")

for p in ax3.patches:
    left, bottom, width, height = p.get_bbox().bounds
    ax3.annotate(round(height, 2), (left+width/2, height), ha='center', color = "black")

ax1.set_title("New Confirmed Cases per 100k in the US")
ax2.set_title("New Confirmed Cases per 100k in Canada")
ax3.set_title("New Confirmed Cases per 100k in South Korea")

ax1.set_ylabel("Cases")
ax2.set_ylabel("Cases")
ax3.set_ylabel("Cases")

ax1.set_ylim(0, covid_merge_df.iloc[:, 6].max()*1.3)
ax2.set_ylim(0, covid_merge_df.iloc[:, 13].max()*1.5)
ax3.set_ylim(0, covid_merge_df.iloc[:, 20].max()*1.8)

ax1.set_xticks(range(len(x_label)))
ax2.set_xticks(range(len(x_label)))
ax3.set_xticks(range(len(x_label)))

ax1.set_xticklabels(x_label, rotation = 0)
ax2.set_xticklabels(x_label, rotation = 0)
ax3.set_xticklabels(x_label, rotation = 0)

plt.savefig("charts/confirmed_cases_100k.png", dpi = 200)
plt.show()

In [ ]:
us_idx = covid_sales_df.loc[covid_sales_df.loc[:, "Country"] == "United States of America"].index
ca_idx = covid_sales_df.loc[covid_sales_df.loc[:, "Country"] == "Canada"].index
kr_idx = covid_sales_df.loc[covid_sales_df.loc[:, "Country"] == "Korea (South)"].index
covid_sales_df = covid_sales_df.reset_index(drop = True)

sr_1 = pd.Series()
sr_1 = sr_1.append(us_sales_1.tail(5).loc[: ,"E-Commerce Growth(%)"], ignore_index = True)
sr_1 = sr_1.append(canada_2.tail(5).loc[:, "E-Commerce Growth(%)"], ignore_index = True)
sr_1 = sr_1.append(korea_2.tail(5).loc[:, "E-Commerce Growth(%)"], ignore_index = True)

sr_2 = pd.Series()
sr_2 = sr_2.append(us_sales_1.tail(5).loc[: ,"E-Commerce Share(%)"], ignore_index = True)
sr_2 = sr_2.append(canada_2.tail(5).loc[:, "E-Commerce Share(%)"], ignore_index = True)
sr_2 = sr_2.append(korea_2.tail(5).loc[:, "E-Commerce Share(%)"], ignore_index = True)

covid_sales_df.loc[:, "E-Commerce Growth(%)"] =  sr_1
covid_sales_df.loc[:, "E-Commerce Share(%)"] =  sr_2
covid_sales_df

In [ ]:
x_axis = covid_sales_df.iloc[:, 2].astype(float)
y_axis = covid_sales_df.iloc[:, 4].astype(float)

print(f"The correlation coefficien between New Cases and E-Commerce Growth(%) is {round(st.pearsonr(x_axis, y_axis)[0], 2)}")

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))

plt.scatter(x_axis, y_axis, c = y_axis, cmap = "viridis", edgecolors = "black")
plt.plot(x_axis, regress_values, "r-")
plt.annotate(line_eq, (0.0010, 0), fontsize = 15, color = "red")
plt.title("New Cases vs. E-Commerce Growth Rate")
plt.grid()
plt.xlabel("New Cases")
plt.ylabel("E-Commerce Growth(%)")
plt.savefig("charts/new_cases_vs._growth_rate.png", dpi = 200)
plt.show()

In [ ]:
us_pop = 328200000
ca_pop = 37590000
kr_pop = 51269185

for idx in covid_sales_df.index:
    if covid_sales_df.loc[idx, "Country"] == "United States of America":
        covid_sales_df.loc[idx, "New Cases per 100000"] = covid_sales_df.loc[idx, "New Cases"] / us_pop * 100000
        covid_sales_df.loc[idx, "Cumulative Cases per 100000"] = covid_sales_df.loc[idx, "Cumulative Cases"] / us_pop * 100000
    elif covid_sales_df.loc[idx, "Country"] == "Canada":
        covid_sales_df.loc[idx, "New Cases per 100000"] = covid_sales_df.loc[idx, "New Cases"] / ca_pop * 100000
        covid_sales_df.loc[idx, "Cumulative Cases per 100000"] = covid_sales_df.loc[idx, "Cumulative Cases"] / ca_pop * 100000
    else:
        covid_sales_df.loc[idx, "New Cases per 100000"] = covid_sales_df.loc[idx, "New Cases"] / kr_pop * 100000
        covid_sales_df.loc[idx, "Cumulative Cases per 100000"] = covid_sales_df.loc[idx, "Cumulative Cases"] / kr_pop * 100000

covid_sales_df

In [ ]:
x_axis = covid_sales_df.iloc[:, 6].astype(float)
y_axis = covid_sales_df.iloc[:, 4].astype(float)

print(f"The correlation coefficient between New Cases per 100000 and E-Commerce Growth(%) is {round(st.pearsonr(x_axis, y_axis)[0], 2)}")

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))

plt.scatter(x_axis, y_axis, c = y_axis, cmap = "viridis", edgecolors = "black")
plt.plot(x_axis, regress_values, "r-")
plt.title("New Cases per 100k vs. E-Commerce Growth Rate")
plt.annotate(line_eq, (50, -5), fontsize = 15, color = "red")
plt.grid()
plt.xlabel("New Cases per 100k")
plt.ylabel("E-Commerce Growth(%)")
plt.savefig("charts/new_cases_per_100k_vs._growth_rate.png", dpi = 200)
plt.show()

In [ ]:
x_axis = covid_sales_df.iloc[:, 7].astype(float)
y_axis = covid_sales_df.iloc[:, 4].astype(float)

print(f"The correlation coefficient between New Cases per 100k and E-Commerce Growth(%) is {round(st.pearsonr(x_axis, y_axis)[0], 2)}")

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))

plt.scatter(x_axis, y_axis, c = y_axis, cmap = "viridis", edgecolors = "black")
plt.plot(x_axis, regress_values, "r-")
plt.title("Cumulative Cases per 100k vs. E-Commerce Growth Rate")
plt.annotate(line_eq, (50, -5), fontsize = 15, color = "red")
plt.grid()
plt.xlabel("Cumulative Cases per 100k")
plt.ylabel("E-Commerce Growth(%)")
plt.savefig("charts/cumulative_cases_per_100k_vs._growth_rate.png", dpi = 200)
plt.show()

In [ ]:
x_axis = covid_sales_df.iloc[:, 2].astype(float)
y_axis = covid_sales_df.iloc[:, 5].astype(float)

print(f"The correlation coefficient between New Cases and E-Commerce Share(%) is {round(st.pearsonr(x_axis, y_axis)[0], 2)}")

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))

plt.scatter(x_axis, y_axis, c = y_axis, cmap = "viridis", edgecolors = "black")
plt.plot(x_axis, regress_values, "r-")
plt.title("New Cases vs. E-Commerce Share")
plt.annotate(line_eq, (50, 20), fontsize = 15, color = "red")
plt.grid()
plt.xlabel("New Cases")
plt.ylabel("E-Commerce Share(%)")
plt.savefig("charts/new_cases_vs._share.png", dpi = 200)
plt.show()

In [ ]:
x_axis = covid_sales_df.iloc[:, 6].astype(float)
y_axis = covid_sales_df.iloc[:, 5].astype(float)

print(f"The correlation coefficient between New Cases per 100k and E-Commerce Share(%) is {round(st.pearsonr(x_axis, y_axis)[0], 2)}")

In [ ]:
(slope, intercept, rvalue, pvalue, stderr) = st.linregress(x_axis, y_axis)
regress_values = x_axis * slope + intercept
line_eq = "y = " + str(round(slope, 2)) + "x + " + str(round(intercept, 2))

plt.scatter(x_axis, y_axis, c = y_axis, cmap = "viridis", edgecolors = "black")
plt.plot(x_axis, regress_values, "r-")
plt.title("New Cases per 100k vs. E-Commerce Share")
plt.annotate(line_eq, (100, 20), fontsize = 15, color = "red")
plt.grid()
plt.xlabel("New Cases per 100k")
plt.ylabel("E-Commerce Share(%)")
plt.savefig("charts/new_cases_per_100k_vs._share.png", dpi = 200)
plt.show()